# Shallow Water Model

2 dimensional shallow water simulattion using discretization

## Partial Differential Equation for Shallow Water Model
\begin{align}
\frac{\partial}{\partial t}u &= -g\frac{\partial}{\partial x}h \\
\frac{\partial}{\partial t}v &= -g\frac{\partial}{\partial y}h \\
\frac{\partial}{\partial t}h + \frac{\partial}{\partial x}(hu) &+ \frac{\partial}{\partial y}(hu) = 0
\end{align}

## Discretization
\begin{align}
u^{n+1}_{ij} &= u^{n}_{ij} - g\frac{\Delta t}{\Delta x}(h_{i,j+1} -h_{i,j})\\
v^{n+1}_{ij} &= v^{n}_{ij} - g\frac{\Delta t}{\Delta y}(h_{i+1,j} -h_{i,j})\\
h^{n+1}_{i,j} = h^{n}_{i,j} &- \frac{\Delta t}{\Delta x}(f^e_{i,j} - f^w_{i,j})  - \frac{\Delta t}{\Delta y}(f^n_{i,j} - f^s_{i,j})\\
\end{align}
where
\begin{align}
f^e_{i,j} = u^+_{i,j}h_{i,j} + u^-_{i,j}h_{i,j+1} \\
f^n_{i,j} = v^+_{i,j}h_{i,j} + v^-_{i,j}h_{i+1,j} \\
f^w_{i,j} = u^+_{i,j}h_{i,j-1} + u^-_{i,j}h_{i,j} \\
f^s_{i,j} = v^+_{i,j}h_{i-1,j} + v^-_{i,j}h_{i,j} \\
\end{align}
and
\begin{align}
u^+_{i,j} = 0.5(u_{i,j} + \mid u_{i,j}\mid) \\
u^-_{i,j} = 0.5(u_{i,j} - \mid u_{i,j}\mid)
\end{align}
and the height is smoothed:
\begin{equation}
h^*_{i,j} = (1-\epsilon)h_{i,j} + \frac{\epsilon}{4}(h_{i-1,j} + h_{i+1,j} + h_{i,j-1} + h_{i,j+1})
\end{equation}

## Setup

In [1]:
using Plots
gr()

Plots.GRBackend()

In [2]:
x = linspace(-50,50,100)
y = linspace(-100,100,100)
dx = x[2] - x[1]
dy = y[2] - y[1]


d = 10
g = 9.81
dt = 0.5 * min(dx, dy) / sqrt(g*d)
t1 = 10
ϵ = 0.1

h0 = d + exp.(-(x'.^2 .+ y.^2)/20)
u0 = zeros(100, 100)
v0 = zeros(100, 100);

In [3]:
function iterate(h, u, v)
    
    h_new = zeros(size(h))
    u_new = zeros(size(u))
    v_new = zeros(size(v))
    
    n1, n2 = size(h)
    for i in 1:n1
        for j in 1:n2
            if j == n2
                u_new[i,j] = 0
            else
                u_new[i,j] = u[i,j] - g*dt/dx * (h[i,j+1] - h[i,j])
            end
            if i == n1
                v_new[i,j] = 0
            else
                v_new[i,j] = v[i,j] - g*dt/dy * (h[i+1,j] - h[i,j])
            end
        end
    end
    
    u_plus = .5 * (u_new + abs.(u_new))
    u_minus = .5 * (u_new - abs.(u_new))
    v_plus = .5 * (v_new + abs.(v_new))
    v_minus = .5 * (v_new - abs.(v_new))
            
    for i in 1:n1
        for j in 1:n2
            if j == n2
                f_e = 0
            else
                f_e = u_plus[i,j] * h[i,j] + u_minus[i,j] * h[i,j+1]
            end
            if i == n1
                f_n = 0
            else
                f_n = v_plus[i,j] * h[i,j] + v_minus[i,j] * h[i+1,j]
            end
            if j == 1
                f_w = 0
            else
                f_w = u_plus[i,j-1] * h[i,j-1] + u_minus[i,j-1] * h[i,j]
            end
            if i == 1
                f_s = 0
            else
                f_s = v_plus[i-1,j] * h[i-1,j] + v_minus[i-1,j] * h[i,j]
            end
            h_new[i,j] = h[i,j] - dt/dx*(f_e - f_w) - dt/dy*(f_n - f_s)
        end
    end
    
    h_filter = zeros(size(h_new))
    for i in 1:n1
        for j in 1:n2
            if j == n2
                h_e = h_new[i,j-1]
            else
                h_e = h_new[i,j+1]
            end
            if i == n1
                h_n = h_new[i-1,j]
            else
                h_n = h_new[i+1,j]
            end
            if j == 1
                h_w = h_new[i,j+1]
            else
                h_w = h_new[i,j-1]
            end
            if i == 1
                h_s = h_new[i+1,j]
            else
                h_s = h_new[i-1,j]
            end
            
            h_filter[i,j] = (1-ϵ) * h_new[i,j] + .25*ϵ*(h_e+h_n+h_w+h_s)
        end
    end
    
    (h_filter, u_new, v_new)
end

iterate (generic function with 1 method)

In [4]:
h = h0
u = u0
v = v0;

In [5]:
hs = []
for i=1:500
    h,u,v = iterate(h,u,v)
    push!(hs, h)
end

In [15]:
dd = @animate for i=1:500
    plot(x,y,hs[i],st=:contour, fill=true, colorbar=false, clims=(9.8,10.2), c=:ice)
    end;

In [16]:
gif(dd, "2d.gif")

INFO: Saved animation to /Users/catethos/Projects/persephonejl/notebooks/2d.gif


Plots.AnimatedGif("/Users/catethos/Projects/persephonejl/notebooks/2d.gif")

In [17]:
ddd = @animate for i=1:500
    plot(x,y,hs[i], st=:surface, zlims=(9.8,10.2), colorbar=false, c=:ice)
    end; 

In [18]:
gif(ddd, "3d.gif")

INFO: Saved animation to /Users/catethos/Projects/persephonejl/notebooks/3d.gif


Plots.AnimatedGif("/Users/catethos/Projects/persephonejl/notebooks/3d.gif")